<a href="https://colab.research.google.com/github/codesigningwiththemachine/dvww2022/blob/main/LanguageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

In [ ]:
#@title ## Install and load requirements
#@markdown ---

#@markdown Install / update Fastai and load other requirements
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab
from fastai.text.all import *
from fastai.collab import *
import pandas as pd


In [ ]:
#@title ## Mount Google Drive
#@markdown ---
#@markdown Mount google drive and load project folder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ## Set Project folder
#@markdown ---
#@markdown Go to project folder

Project_Folder = '/content/drive/MyDrive/Language/' #@param {type:"string"}
%cd $Project_Folder

from pathlib import Path
path = Path(Project_Folder)

# **Language Model**

In [ ]:
#@title ## Load Dataset
LM_dataset = "text.csv" #@param {type:"string"}

#Pandas dataframe from csv file
df = pd.read_csv(LM_dataset)
df.head()

In [ ]:
#@title ## Convert Dataset
#@markdown ---
#@markdown Convert data into a dataloader object to be used by fastai.
#@markdown Data is prepared for language model traning. tokenizing words and preparing a required dependant variable for self supervised learning.

dls_lm = TextDataLoaders.from_df(df, text_col='text', is_lm=True, valid_pct=0.1)
dls_lm.show_batch(max_n=2)

In [ ]:
#@title ## Create and train a language model
#@markdown ---

#@markdown Using language_model_learner to prepare and train a language model based on our data. A pretrained RNN is used as a based AWD_LSTM 

learn = language_model_learner(dls_lm, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, Perplexity()]).to_fp16()
# to_fp16() puts the Learner in mixed precision, which is going to help speed up training on GPUs
# Reduce overfitting with drop mult

#is necessary find the right learing rate
#learn.lr_find()
learn.fit_one_cycle(1, 3e-2)


In [ ]:
#@title ## Test our language model
#@markdown ---

Phrase = "I want to go to your" #@param {type:"string"}
learn.predict(Phrase)

In [ ]:
#@title ## Save our model
#@markdown ---

Model = "test" #@param {type:"string"}
learn.save(Model)

In [ ]:
#@title ## Load our model
#@markdown ---

Model = "test" #@param {type:"string"}
learn.load(Model)

In [ ]:
#@title ## Improve model accuracy
#@markdown ---
#@markdown Unfreeze model and train for longer epochs

epochs =  3#@param {type:"number"}

learn.unfreeze()
#learn.lr_find()
learn.fit_one_cycle(epochs, 1e-2)

In [ ]:
#@title ## Test our language model
#@markdown ---

#@markdown Using language_model_learner to prepare and train a language model based on our data. A pretrained RNN is used as a based AWD_LSTM 

Phrase = "I am " #@param {type:"string"}
Words =  16#@param {type:"number"}
Sentences =  1#@param {type:"number"}
Temperature = 0.8 #@param {type:"number"}

for i in range(Sentences):
  print(learn.predict(Phrase, Words, temperature=0.75))


In [ ]:
#@title ## Save our encoder
#@markdown ---
#@markdown Save the model model not including the final layer (encoder)
learn.save_encoder('encoder')

# **Classifier**

In [ ]:
#@title ## Load Dataset
CL_dataset = "/content/drive/MyDrive/Language/tweets_class.csv" #@param {type:"string"}

#Pandas dataframe from csv file
df = pd.read_csv(CL_dataset)
df.head()


In [ ]:
#@title ## Convert Dataset
#@markdown ---
#@markdown Convert data into a dataloader object to be used by fastai.

dls_clas = TextDataLoaders.from_df(df, text_col='text', label_col='sentiment', text_vocab=dls_lm.vocab)
dls_clas.show_batch(max_n=2)

In [ ]:
#@title ## Prepare model and load our encoder
#@markdown ---

learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()
Encoder = "encoder" #@param {type:"string"}
learn.load_encoder(Encoder)

In [ ]:
#@title ## Train our language classifier
#@markdown ---

#@markdown Using fit_one_cycle to train our model for an epoch

#learn.lr_find()
learn.fit_one_cycle(1, 3e-2)



In [ ]:
#@title ## Improve our model
#@markdown ---

#@markdown Gradually unfreeze and continue training 

learn.freeze_to(-2)
#learn.lr_find()
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

learn.freeze_to(-3)
#learn.lr_find()
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

learn.unfreeze()
#learn.lr_find()
learn.fit_one_cycle(4, slice(2e-2/(2.6**4),2e-2))

In [ ]:
#@title ## Test our Emotion Classifier
#@markdown ---

#@markdown Using language_model_learner to prepare and train a language model based on our data. A pretrained RNN is used as a based AWD_LSTM 

Phrase = "hello world" #@param {type:"string"}
print(learn.predict(Phrase)[0])

In [ ]:
#@title ## Save our model
#@markdown ---

Model = "classifier" #@param {type:"string"}
learn.save(Model)

In [ ]:
#@title ## Load our model
#@markdown ---

Model = "classifier" #@param {type:"string"}
learn = learn.load(Model)

# **Deploy Model**

In [ ]:
%%capture
#@title ## Install and load requirements
#@markdown ---

#@markdown Install and setup requirements to serve model online

! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab
!pip install flask
!pip install flask-ngrok
!pip install flask-bootstrap
!pip install jsonpickle
!pip install -U flask-cors

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!./ngrok authtoken 22aZcoUzJr4EMX5EXzWI1kbzawH_55G5fBvzCn8mtyhW9Woai

In [ ]:
#@title ## Run web server 
#@markdown ---

#@markdown 

from flask_ngrok import run_with_ngrok
from flask import Flask, request, Response
from flask_cors import CORS

import jsonpickle
import numpy as np
import cv2
import base64
from io import BytesIO
from PIL import Image



# Initialize the Flask application
app = Flask(__name__)
CORS(app)

# route http posts to this method
run_with_ngrok(app)
@app.route('/api/predict', methods=['POST'])
def test():
    content = request.json
    text = content['text']
    prediction = learn.predict(text)[0]
    #print(prediction)
    response = {'Prediction': '{}'.format(prediction)}
    response_pickled = jsonpickle.encode(response)
    return Response(response=response_pickled, status=200, mimetype="application/json")



if __name__ == '__main__':
   app.run()

In [ ]:
#@title Our url

path = "https://a5cd-34-86-74-216.ngrok.io/api/predict" #@param {type: 'string'}

# **URL to test: [Prediction](https://editor.p5js.org/codesigningwiththemachine/sketches/TMwvIa9BY)**